In [ ]:
# %matplotlib inline #notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것 

import numpy
import pandas
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

#setting the seed for reproducibility
#seed = 10110
seed = 12061204
numpy.random.seed(seed)

#폴더의 파일목록 불러오기
import os
path_dir = 'C:\/test/Deepchem toxcast_data all/Seperated'
file_list = os.listdir(path_dir)
input_files = []
i = 0

for item in file_list :
        input_files.append(item)
        #print(item)
        
def create_deep_learning_model():
    model = Sequential()
    #첫번째 인자 : 출력 뉴런의 수 , input_dim : 입력 뉴런의 수, activation : 활성화 함수.
    model.add(Dense(2048, input_dim=2048, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model    

def main(input_name):
    dataframe = pandas.read_csv('C:\/test/Deepchem toxcast_data all/Seperated/'+input_name)
    print(dataframe.head)
    mols = []
    fps = []
    
    for index, row in dataframe.iterrows():
        mol = Chem.MolFromSmiles(row['SMILES'])
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
        mols.append(mol)
        fps.append(fp)
    
    np_fps = []
    for fp in fps:
        arr = numpy.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, arr)
        np_fps.append(arr)
        
    np_fps_array = numpy.array(np_fps)
    
#     my_string = input_name
#     Type = my_string.split('(')[0]
    enc_y = dataframe['Type']
    
    sm = SMOTE(random_state=12, ratio = 'minority')
    x_train_res, y_train_res = sm.fit_sample(np_fps_array, enc_y) #결과데이터

    # evaluate model with standardized dataset
    #estimator = KerasClassifier(build_fn=create_deep_learning_model, nb_epoch=100, batch_size=5, verbose=0)
    estimator = KerasClassifier(build_fn=create_deep_learning_model, nb_epoch=100, batch_size=5)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    #results = cross_val_score(estimator, np_fps_array, enc_y, cv=kfold)
    results = cross_val_score(estimator, x_train_res, y_train_res, cv=kfold)
    #https://datascienceschool.net/view-notebook/266d699d748847b3a3aa7b9805b846ae/
    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

    y_pred = cross_val_predict(estimator, x_train_res, y_train_res, cv=kfold)
    conf_mat = confusion_matrix(y_train_res, y_pred)
    conf_mat


    model = create_deep_learning_model()
    hist = model.fit(x_train_res, y_train_res, epochs=5, batch_size=5)
    model.save("C:\/test/ppar-gamma-model/trained_models/models_2/"+input_name+'_model' + '(' + hist.history['acc'][-1] + ')'+'.h5')

if __name__ == "__main__":
#     for input_name in input_files :
        main('APR_Hepat_Apoptosis_24hr_up(input).csv')    